In [ ]:
import time
import random
import requests
import json
import re
import os

API_KEY = os.getenv("ANTHROPIC_API_KEY")
BASE_URL = "https://api.anthropic.com/v1/messages"
MODEL_NAME = "claude-3-sonnet-20240229"
MAX_RETRIES = 5


In [ ]:
def call_claude(prompt):
    retries = 0
    while retries < MAX_RETRIES:
        try:
            headers = {
                "x-api-key": API_KEY,
                "Content-Type": "application/json",
                "anthropic-version": "2023-06-01"
            }

            # Correct message structure for the Messages API
            data = {
                "model": MODEL_NAME,
                "messages": [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "max_tokens": 50,
                "temperature": 0.7,
                "stop_sequences": ["\n\nHuman:"]
            }

            response = requests.post(BASE_URL, headers=headers, data=json.dumps(data))

            if response.status_code == 200:
                result = response.json()

                # Correctly extract the response content
                if "content" in result and isinstance(result["content"], list):
                    for item in result["content"]:
                        if item.get("type") == "text":
                            return item.get("text", "Erreur: Réponse sans texte").strip()
                    return "Erreur: Pas de contenu texte trouvé"
                else:
                    print(f"Unexpected response format: {result}")
                    return "Error: No valid response found"
            else:
                print(f"Error: {response.status_code} - {response.json().get('message', 'Unknown error')}")
                print(f"Response Content: {response.text}")
                retries += 1
                time.sleep(2 ** retries + random.uniform(0, 1))  # Exponential backoff

        except requests.RequestException as e:
            print(f"Request error: {e}")
            retries += 1
            time.sleep(2 ** retries + random.uniform(0, 1))  # Exponential backoff

    return "Error: Maximum retry attempts reached."


In [ ]:
def is_paragraph(text, output_dir, index):
    """
    Vérifie si le texte donné ressemble à un paragraphe en utilisant l'API Claude.
    Sauvegarde également la réponse dans un fichier dédié.
    """
    prompt = f"Does this text look like a paragraph or a part or a clause from a contract? Say 'Yes' or 'No'. Texte: {text}"
    answer = call_claude(prompt)

    # Sauvegarder la réponse dans un fichier
    responses_dir = os.path.join(output_dir, "reponses_api")
    if not os.path.exists(responses_dir):
        os.makedirs(responses_dir)

    response_filename = os.path.join(responses_dir, f"response_{index}.txt")
    with open(response_filename, 'w', encoding='utf-8') as response_file:
        response_file.write(f"Texte: {text}\nRéponse: {answer}\n")

    return answer.lower().startswith("yes")


In [ ]:

def segment_markdown(file_path, output_dir):
    # Créer le dossier de sortie si nécessaire
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Lire le contenu du fichier Markdown
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Détecter les chapitres avec les modèles suivants :
    # - En-têtes Markdown (#, ##, ###, etc.)
    # - Numérotation simple : "1.", "2.", "3."
    # - Numérotation imbriquée : "3.1.", "3.1.1."
    # - Numérotation par lettres : "(a)", "(b)", "(c)"
    # - Numérotation avec tiret : "- 1.", "- 2.1.", "- 3.1.1."
    chapter_pattern = r'(^#{1,6}\s.*|^-?\s?\d+(\.\d+)*\.\s.*|^-?\s?\([a-z]\)\s.*)'

    # Diviser le contenu en segments basés sur les en-têtes
    chapters = re.split(chapter_pattern, content, flags=re.MULTILINE)
    chapters = [chapter.strip() for chapter in chapters if chapter and chapter.strip()]
    
    current_header = None
    # add an number at the beginning of the file name for each chapter
    for i, chapter in enumerate(chapters):        # Vérifier si le segment correspond à un en-tête
        if re.match(chapter_pattern, chapter) and is_paragraph(chapter, output_dir, i):
            # Nettoyer le nom du fichier en remplaçant les espaces et caractères spéciaux
            safe_header = re.sub(r'[^a-zA-Z0-9_\.]', '_', chapter)[:50]
            current_header = safe_header
            file_name = f"{i} : " + f"{safe_header}.txt"
            file_path = os.path.join(output_dir, file_name)
            with open(file_path, 'w', encoding='utf-8') as chapter_file:
                chapter_file.write(f"{chapter}\n")
        elif current_header:
            # Ajouter le contenu au fichier du chapitre en cours
            with open(file_path, 'a', encoding='utf-8') as chapter_file:
                chapter_file.write(chapter + '\n')

    # print(f"Segmentation terminée pour : {file_path}")


In [ ]:

def process_all_markdowns(base_dir, output_base):
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            # Vérifier si le fichier est un .md
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                # Extraire le nom du dossier parent pour structurer les résultats
                folder_name = os.path.basename(root)
                # Créer le dossier de sortie pour ce fichier spécifique
                output_dir = os.path.join(output_base, folder_name)
                segment_markdown(file_path, output_dir)
    
    print("Traitement terminé pour tous les fichiers Markdown.")


In [ ]:

# Configuration des répertoires
base_dir = '../contracts_3_redacted'
output_base = './fichiers segmentés'

# Exécuter le traitement pour tous les fichiers Markdown dans tous les sous-dossiers
process_all_markdowns(base_dir, output_base)

# i want to segment only for one file: ACM_2016_Agreement.md located in ../contracts_3_redacted/ACM_2016_Agreement
# file_path_test = '../contracts_3_redacted/ACM_2016_Agreement/ACM_2016_Agreement.md'
# output_dir_test = './ACM_2016_Agreement'
# segment_markdown(file_path_test, output_dir_test)